<a href="https://colab.research.google.com/github/MdelaVilla/MORS/blob/main/Sesion_5_MORS_Clasificaci%C3%B3n_binaria_con_Transformers_y_corpus_HateEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cuaderno hecho para el grupo **I2C** por los profesores Jacinto Mata y Victoria Pachón.

*Tras cada sección aparece un comentario aclaratoio que explica línea a línea el proceso seguido.*

## **1. Importar librerías**

In [19]:
!pip install datasets transformers
!pip install --upgrade datasets transformers

## **Explicación del Cuaderno Paso a Paso**

### **1. Importar Librerías**

Esta sección instala y carga las librerías necesarias para el proyecto. Es crucial tener las versiones correctas de las librerías `datasets` y `transformers`, ya que son fundamentales para el procesamiento de datos y la construcción del modelo BERT.

*   `!pip install transformers==4.30.0 datasets==2.10.0`: Este comando se asegura de instalar versiones específicas de las librerías `transformers` y `datasets`. Es importante usar versiones compatibles para evitar problemas de compatibilidad y errores que puedan surgir de cambios en la API entre diferentes versiones. La versión `4.30.0` de `transformers` y `2.10.0` de `datasets` son versiones estables y conocidas por funcionar bien juntas en tareas de procesamiento de lenguaje natural con modelos como BERT.

In [22]:
import pandas as pd
import re
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report

*   `import pandas as pd`: Importa la librería `pandas`, que es una herramienta muy potente para la manipulación y análisis de datos en formato de tablas (DataFrames).
*   `import re`: Importa el módulo `re`, que proporciona operaciones con expresiones regulares, útil para el preprocesamiento de texto.
*   `from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback`: Importa componentes específicos de la librería `transformers`:
    *   `BertTokenizer`: Se encarga de convertir el texto en un formato que el modelo BERT pueda entender.
    *   `BertForSequenceClassification`: La clase principal para el modelo BERT adaptado a tareas de clasificación de secuencias.
    *   `Trainer`: Una clase de utilidad que facilita el bucle de entrenamiento, evaluación y predicción de modelos de Transformers.
    *   `TrainingArguments`: Define los hiperparámetros y configuraciones para el entrenamiento del modelo.
    *   `EarlyStoppingCallback`: Una herramienta para detener el entrenamiento si la métrica de evaluación no mejora después de un cierto número de épocas, evitando el sobreajuste.
*   `import torch`: Importa la librería `torch`, que es el framework de aprendizaje profundo subyacente utilizado por `transformers`.
*   `from sklearn.model_selection import train_test_split`: Importa la función `train_test_split` de `scikit-learn`, que se utiliza para dividir el conjunto de datos en subconjuntos de entrenamiento y prueba.
*   `from torch.utils.data import DataLoader, Dataset`: Importa clases de `PyTorch` para trabajar con conjuntos de datos y cargadores de datos, que permiten manejar grandes volúmenes de datos de manera eficiente durante el entrenamiento.
*   `from sklearn.metrics import classification_report`: Importa la función `classification_report` de `scikit-learn` para generar un informe detallado de las métricas de clasificación (precisión, recall, f1-score) del modelo.

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **2. Cargar Dataset**

In [24]:
# Cargamos el dataset desde un archivo CSV
df = pd.read_csv('/content/drive/MyDrive/corpus/HateEval/train_es.tsv', sep='\t', usecols =["text","HS"])
# Mostramos los primeros registros para verificar que los datos se cargaron correctamente
print(df.head())

                                                text  HS
0  Easyjet quiere duplicar el número de mujeres p...   1
1  El gobierno debe crear un control estricto de ...   1
2  Yo veo a mujeres destruidas por acoso laboral ...   0
3  — Yo soy respetuoso con los demás, sólamente l...   0
4  Antonio Caballero y como ser de mal gusto e ig...   0


### **2. Cargar Dataset**

Esta sección se enfoca en cargar el conjunto de datos principal para el entrenamiento y evaluación del modelo. El dataset se carga desde un archivo CSV ubicado en Google Drive, lo que requiere montar primero la unidad de Drive en Colab.

*   `from google.colab import drive` y `drive.mount('/content/drive')`: Estas líneas son estándar en Google Colab para permitir que el cuaderno acceda a los archivos almacenados en tu Google Drive. Al ejecutar `drive.mount()`, se te pedirá que autorices el acceso.

*   `df = pd.read_csv('/content/drive/MyDrive/corpus/HateEval/train_es.tsv', sep='\t', usecols=["text","HS"])`: Aquí se lee el archivo `train_es.tsv` (que es un archivo de valores separados por tabulaciones, `sep='\t'`) utilizando `pandas`. Se especifican las columnas `"text"` y `"HS"` (que probablemente representa 'Hate Speech' o similar) para cargar solo los datos relevantes. `df` es el DataFrame de pandas que contendrá nuestros datos.

*   `print(df.head())`: Después de cargar los datos, esta línea muestra las primeras cinco filas del DataFrame para que puedas verificar que la carga se realizó correctamente y que los datos tienen el formato esperado.

*   `df.columns = ['text', 'label']`: Se renombran las columnas del DataFrame para que sean más descriptivas y coherentes con la terminología de los modelos de clasificación. `text` para el contenido textual y `label` para la categoría o etiqueta (en este caso, si es discurso de odio o no).

*   `value_counts = df['label'].value_counts()`: Esta línea calcula cuántas instancias hay de cada valor único en la columna `label`. Es una forma rápida de ver la distribución de las clases en el dataset (por ejemplo, cuántos ejemplos son 'discurso de odio' y cuántos no lo son). Esto es útil para identificar posibles desequilibrios de clases.

In [25]:
df.columns = ['text', 'label']
df

,text,label
0,Easyjet quiere duplicar el número de mujeres p...,1
1,El gobierno debe crear un control estricto de ...,1
2,Yo veo a mujeres destruidas por acoso laboral ...,0
3,"— Yo soy respetuoso con los demás, sólamente l...",0
4,Antonio Caballero y como ser de mal gusto e ig...,0
...,...,...
4464,@miriaan_ac @Linaveso_2105 @HumildesSquad_ CÁL...,1
4465,"@IvanDuque presidente en Cúcuta , tenemos prob...",1
4466,- Callaté Visto Que Te Dejo En Puta🎤🎶,0
4467,-¿porque los hombres se casan con las mujeres?...,1


In [26]:
#Vemos cuantas instancias hay de cada tipo en el dataset

value_counts = df['label'].value_counts()
value_counts

,count
label,
0,2631
1,1838


## **3. Preprocesado básico**

In [27]:
def to_lowercase(tweet):
    """Takes a string and converts all characters to lowercase"""
    return tweet.lower()

In [28]:
# Aplicamos el preprocesamiento al texto de nuestro DataFrame
df['text'] = df['text'].apply(to_lowercase)

Comprobamos si ha funcionado el preprocesado

In [29]:
#Mostramos dos filas

print(df.iloc[17]['text'])
print(df.iloc[24]['text'])

vamoooo la puta madre se lo merecía tanto esfuerzo y haber pasado tantos teléfonos necesitábamos este descanso #soltartenoestáenmisplanesmica
si cualquier cosa es violación o acoso, se minimizan la violación y el acoso. por lo tanto, pierden las víctimas de las mierdas que se dedican a violar y/o acosar.


## **3. Preprocesado básico**

Esta sección se centra en las primeras etapas de limpieza del texto, que son fundamentales para preparar los datos antes de la tokenización y el entrenamiento del modelo. Un preprocesamiento adecuado puede mejorar significativamente el rendimiento del modelo.

*   `def to_lowercase(tweet): return tweet.lower()`: Define una función simple llamada `to_lowercase` que toma una cadena de texto (`tweet`) y devuelve la misma cadena convertida completamente a minúsculas. Esta es una práctica común en el procesamiento de lenguaje natural para estandarizar el texto y asegurar que, por ejemplo, 'Hola' y 'hola' se traten como la misma palabra.

*   `df['text'] = df['text'].apply(to_lowercase)`: Aplica la función `to_lowercase` a cada entrada de la columna `text` de tu DataFrame `df`. El método `.apply()` de pandas es muy útil para aplicar una función a todos los elementos de una serie (columna) de un DataFrame.

*   **Comprobación del preprocesado:** Las líneas siguientes en el cuaderno (`print(df.iloc[17]['text'])` y `print(df.iloc[24]['text'])`) están diseñadas para mostrar cómo ha quedado el texto después de aplicar la conversión a minúsculas, permitiendo verificar que el preprocesamiento se realizó correctamente.

## **4. Conversión a Dataset y Tokenización**

Usamos el tokenizador de BERT para convertir el texto en vectores que pueda entender el modelo, truncando o rellenando hasta la longitud máxima establecida.

In [30]:
# Cargar el tokenizador de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convertir el DataFrame a Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)

# Definir la función de tokenización
def tokenize_function(example):
    return tokenizer(example["text"], padding='max_length', truncation=True, max_length=128)

# Aplicar la tokenización al Dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Convertir las columnas necesarias a formato tensor de PyTorch
tokenized_dataset = tokenized_dataset.with_format("torch", columns=["input_ids", "attention_mask", "label"])

# Verificar el dataset tokenizado
print(tokenized_dataset[0])

Map:   0%|          | 0/4469 [00:00<?, ? examples/s]

{'label': tensor(1), 'input_ids': tensor([  101,  3733, 15759, 21864,  7869,  4241, 24759,  5555,  2099,  3449,
        16371,  5017,  2080,  2139, 14163, 20009,  2229,  4405,  2080,  1005,
        12297,  2015, 10722, 11498,  9706,  2906, 10010,  3449, 20704,  3258,
         1012,  1012,  8299,  1024,  1013,  1013,  1056,  1012,  2522,  1013,
         4805, 11231, 13687,  2213,  2692,  2683,  2595,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  

## **4. Conversión a Dataset y Tokenización**

En esta sección, el texto se prepara para ser introducido en el modelo BERT. Esto implica dos pasos principales: la conversión del DataFrame de pandas a un objeto `Dataset` de Hugging Face y la tokenización del texto.

*   `tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')`: Aquí se carga un tokenizador pre-entrenado específico para el modelo `bert-base-uncased`. El tokenizador es esencial para dividir el texto en "tokens" (palabras o subpalabras), añadir tokens especiales (`[CLS]`, `[SEP]`), y convertir estos tokens en IDs numéricos que el modelo entiende. `bert-base-uncased` se refiere a la versión base de BERT, que no distingue entre mayúsculas y minúsculas.

*   `from datasets import Dataset` y `dataset = Dataset.from_pandas(df)`: La librería `datasets` de Hugging Face es muy eficiente para manejar grandes conjuntos de datos en el contexto del aprendizaje profundo. Esta línea convierte el DataFrame de pandas (`df`) que contiene nuestro texto y etiquetas en un objeto `Dataset` de Hugging Face, lo cual es el formato preferido para trabajar con los modelos de la librería `transformers`.

*   `def tokenize_function(example): return tokenizer(example["text"], padding='max_length', truncation=True, max_length=128)`: Se define una función para tokenizar el texto. Esta función toma un ejemplo del dataset, extrae el texto (`example["text"]`), y lo tokeniza usando el `tokenizer` cargado. Los argumentos importantes son:
    *   `padding='max_length'`: Asegura que todas las secuencias tokenizadas tengan la misma longitud, rellenando con ceros si son más cortas que `max_length`.
    *   `truncation=True`: Si una secuencia es más larga que `max_length`, se trunca para ajustarse.
    *   `max_length=128`: Define la longitud máxima de la secuencia de tokens. Este valor es importante porque los modelos BERT tienen un límite en la longitud de entrada.

*   `tokenized_dataset = dataset.map(tokenize_function, batched=True)`: Esta línea aplica la función `tokenize_function` a todo el `dataset`. El argumento `batched=True` permite que el tokenizador procese múltiples ejemplos a la vez, lo que es mucho más eficiente.

*   `tokenized_dataset = tokenized_dataset.with_format("torch", columns=["input_ids", "attention_mask", "label"])`: Después de la tokenización, se configuran las columnas necesarias (`input_ids`, `attention_mask`, y `label`) para que estén en formato `torch.Tensor`. Esto es fundamental ya que `torch` es el backend de PyTorch que utiliza `transformers`.

*   `print(tokenized_dataset[0])`: Finalmente, se imprime el primer ejemplo del `tokenized_dataset` para verificar cómo se ve un ejemplo después de la tokenización, mostrando los `input_ids` (IDs numéricos de los tokens), la `attention_mask` (para indicar qué tokens son reales y cuáles son relleno) y la `label`.

## **5. División en Conjuntos de Entrenamiento, validación y test**

Dividimos el conjunto de entrenamiento en 65% train, 15% validación y 20% test

In [31]:
# Dividir en entrenamiento (80%) y test (20%)
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_data = train_test_split['train']
test_data = train_test_split['test']


# Dividir el conjunto de entrenamiento en entrenamiento (70%) y validación (30%)
train_valid_split = train_data.train_test_split(test_size=0.3, seed=42)
train_data = train_valid_split['train']
valid_data = train_valid_split['test']

In [32]:
train_data

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2502
})

In [33]:
valid_data

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1073
})

In [34]:
test_data

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 894
})

## **5. División en Conjuntos de Entrenamiento, validación y test**

Esta sección es crucial para asegurar una evaluación imparcial del modelo. El dataset tokenizado se divide en tres partes:

*   **Conjunto de Entrenamiento (Train Set):** Utilizado para entrenar el modelo. El modelo aprende de estos datos.
*   **Conjunto de Validación (Validation Set):** Utilizado durante el entrenamiento para ajustar hiperparámetros y detectar el sobreajuste. El rendimiento del modelo en este conjunto guía las decisiones durante el entrenamiento.
*   **Conjunto de Prueba (Test Set):** Utilizado para la evaluación final del modelo, una vez que el entrenamiento ha concluido. Estos datos son completamente nuevos para el modelo y proporcionan una estimación realista de su rendimiento en datos no vistos.

El proceso de división se realiza en dos pasos:

1.  **División inicial en Entrenamiento y Prueba:**
    *   `train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)`: El `tokenized_dataset` se divide por primera vez. El `test_size=0.2` indica que el 20% de los datos se asignará al conjunto de prueba, y el 80% restante al conjunto de entrenamiento inicial. `seed=42` asegura que la división sea reproducible.
    *   `train_data = train_test_split['train']`: Los datos del 80% se asignan a `train_data`.
    *   `test_data = train_test_split['test']`: Los datos del 20% se asignan a `test_data`.

2.  **División del Conjunto de Entrenamiento inicial en Entrenamiento y Validación:**
    *   `train_valid_split = train_data.train_test_split(test_size=0.3, seed=42)`: El `train_data` (el 80% original) se divide nuevamente. Esta vez, el `test_size=0.3` significa que el 30% de este 80% (lo que representa el 24% del dataset total: 0.3 * 0.8 = 0.24) se utilizará para validación. El `seed=42` se mantiene para reproducibilidad.
    *   `train_data = train_valid_split['train']`: El 70% del `train_data` inicial (56% del total: 0.7 * 0.8 = 0.56) se convierte en el conjunto de entrenamiento final.
    *   `valid_data = train_valid_split['test']`: El 30% del `train_data` inicial (24% del total) se convierte en el conjunto de validación.

Al final, los datos se dividen aproximadamente en:
*   **Entrenamiento:** 56% del total (originalmente 65% + 15% + 20%, se reajusta a 56% train, 24% validation, 20% test).
*   **Validación:** 24% del total.
*   **Prueba:** 20% del total.

Las impresiones de `train_data`, `valid_data`, y `test_data` que ves en las celdas siguientes (`Ntz5qTO3Rs0F`, `Ib1Nto-yRxtV`, `B9C8oxprRzfo`) muestran un resumen de cada `Dataset`, incluyendo las características disponibles y el número de filas en cada conjunto, confirmando el resultado de las divisiones.

## **6. Carga del modelo BERT para clasificación**

Cargamos el modelo preentrenado BertForSequenceClassification con dos etiquetas de salida.

In [35]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## **6. Carga del modelo BERT para clasificación**

Esta sección se encarga de cargar el modelo BERT que utilizaremos para la tarea de clasificación de secuencias.

*   `model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)`: Esta línea es clave. Aquí:
    *   `BertForSequenceClassification`: Es la clase de modelo de la librería `transformers` específicamente diseñada para tareas de clasificación de secuencias utilizando BERT.
    *   `from_pretrained('bert-base-uncased')`: Carga los pesos pre-entrenados del modelo `bert-base-uncased`. `bert-base-uncased` es una versión de BERT que ha sido entrenada con un gran corpus de texto y no distingue entre mayúsculas y minúsculas (de ahí 'uncased'). Utilizar un modelo pre-entrenado es una práctica común en NLP, ya que permite aprovechar el conocimiento lingüístico adquirido durante su pre-entrenamiento en una tarea general de comprensión del lenguaje.
    *   `num_labels=2`: Indica que el modelo se está configurando para una tarea de clasificación binaria, donde hay dos posibles etiquetas de salida (en nuestro caso, probablemente 0 para 'no discurso de odio' y 1 para 'discurso de odio'). El modelo ajustará su capa de clasificación final para tener dos neuronas de salida.

## **7. Configuración de parámetros de entrenamiento**

Configuramos los parámetros de entrenamiento, incluyendo el número de épocas y el tamaño del lote.

In [39]:
training_args = TrainingArguments(
    output_dir='./results',       # Carpeta para guardar los resultados
    num_train_epochs=4,           # Número de épocas
    per_device_train_batch_size=8,  # Tamaño del lote de entrenamiento
    per_device_eval_batch_size=8,  # Tamaño del lote de evaluación
    eval_strategy="epoch",   # Evaluar al final de cada época
    logging_dir='./logs',
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',
    save_strategy = 'epoch'
)

## **7. Configuración de parámetros de entrenamiento**

Esta sección define los argumentos y configuraciones que guiarán el proceso de entrenamiento del modelo BERT.

*   `training_args = TrainingArguments(...)`: Se crea un objeto `TrainingArguments` que encapsula todos los hiperparámetros y la configuración para el `Trainer`. Analicemos los argumentos clave:
    *   `output_dir='./results'`: Especifica el directorio donde se guardarán los resultados del entrenamiento, como los checkpoints del modelo y las métricas de evaluación.
    *   `num_train_epochs=4`: Define el número total de épocas (pasadas completas sobre el conjunto de datos de entrenamiento) que el modelo realizará.
    *   `per_device_train_batch_size=8`: Establece el tamaño del lote para el entrenamiento en cada dispositivo (GPU/CPU). Lotes más pequeños pueden requerir más tiempo de entrenamiento pero pueden ser útiles con memoria limitada o para mejorar la generalización.
    *   `per_device_eval_batch_size=8`: Establece el tamaño del lote para la evaluación en cada dispositivo.
    *   `eval_strategy="epoch"`: Indica que la evaluación del modelo se realizará al final de cada época de entrenamiento. También podría ser por pasos (`"steps"`).
    *   `logging_dir='./logs'`: Especifica el directorio donde se guardarán los logs del entrenamiento, útiles para monitorear el progreso con herramientas como TensorBoard.
    *   `load_best_model_at_end=True`: Configura el `Trainer` para que, al finalizar el entrenamiento, cargue automáticamente la versión del modelo que obtuvo el mejor rendimiento en el conjunto de validación (según `metric_for_best_model`).
    *   `metric_for_best_model='f1'`: Define la métrica que el `Trainer` usará para determinar cuál es el "mejor" modelo. En este caso, es el F1-score, que es una métrica común para problemas de clasificación, especialmente cuando hay desequilibrio de clases. El `Trainer` intentará maximizar esta métrica.
    *   `save_strategy="epoch"`: Especifica que el modelo se guardará (checkpoint) al final de cada época. Esto permite recuperar el entrenamiento si se interrumpe y comparar el rendimiento de diferentes épocas.

## **8. Entrenamiento y evaluación del modelo**

Usamos Trainer para entrenar y evaluar el modelo con el conjunto de entrenamiento y el de validación.

In [40]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Definir la función compute_metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=-1)

    # Calcular las métricas usando las predicciones y etiquetas
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }




trainer = Trainer(
    model=model,                         # Modelo a entrenar
    args=training_args,                  # Argumentos de entrenamiento
    train_dataset=train_data,            # Conjunto de entrenamiento
    eval_dataset=valid_data,             # Conjunto de evaluación
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

# Ejecutar el entrenamiento
trainer.train()

# Evaluar el modelo
trainer.evaluate()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mdelavilla to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.629910,0.689655,0.649025,0.529545,0.583229
2,0.598800,0.508991,0.792171,0.780362,0.686364,0.730351
3,0.598800,0.517332,0.785648,0.738636,0.738636,0.738636
4,0.403500,0.828000,0.791240,0.762136,0.713636,0.737089


{'eval_loss': 0.5173322558403015,
 'eval_accuracy': 0.7856477166821995,
 'eval_precision': 0.7386363636363636,
 'eval_recall': 0.7386363636363636,
 'eval_f1': 0.7386363636363636,
 'eval_runtime': 7.3043,
 'eval_samples_per_second': 146.9,
 'eval_steps_per_second': 18.482,
 'epoch': 4.0}

## **8. Entrenamiento y evaluación del modelo**

En esta sección, se configura y ejecuta el proceso de entrenamiento y evaluación del modelo BERT utilizando la clase `Trainer` de la librería `transformers`.

*   `from sklearn.metrics import accuracy_score, precision_recall_fscore_support`:
    Se importan las métricas necesarias de `scikit-learn` para evaluar el rendimiento del modelo.

*   `def compute_metrics(eval_pred): ...`:
    Se define una función `compute_metrics` que el `Trainer` utilizará para calcular métricas de evaluación durante el entrenamiento y la evaluación. Esta función toma las predicciones (`predictions`) y las etiquetas reales (`labels`) del modelo. Dentro de la función:
    *   `preds = predictions.argmax(axis=-1)`: Convierte las logits (salidas crudas del modelo antes de la función de activación) en predicciones de clases, seleccionando la clase con la probabilidad más alta.
    *   `accuracy = accuracy_score(labels, preds)`: Calcula la precisión, que es la proporción de predicciones correctas.
    *   `precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')`: Calcula la precisión, recall y F1-score. `average='binary'` se usa para problemas de clasificación binaria, donde se calcula la métrica para la clase positiva (generalmente 1).
    *   La función devuelve un diccionario con estas métricas.

*   `trainer = Trainer(...)`:
    Se inicializa el objeto `Trainer` con los siguientes parámetros:
    *   `model=model`: El modelo `BertForSequenceClassification` que hemos cargado previamente.
    *   `args=training_args`: Los argumentos de entrenamiento definidos en la sección anterior (`TrainingArguments`).
    *   `train_dataset=train_data`: El conjunto de datos de entrenamiento.
    *   `eval_dataset=valid_data`: El conjunto de datos de validación, que se usará para evaluar el modelo durante el entrenamiento y para la parada temprana.
    *   `compute_metrics=compute_metrics`: La función que hemos definido para calcular las métricas.
    *   `callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]`: Se añade una callback de parada temprana. Esto significa que si la métrica de evaluación (en este caso, 'f1' como se definió en `TrainingArguments`) no mejora durante 3 épocas consecutivas, el entrenamiento se detendrá automáticamente para evitar el sobreajuste.

*   `trainer.train()`:
    Inicia el proceso de entrenamiento del modelo. El `Trainer` gestionará el bucle de entrenamiento, la evaluación periódica y el guardado del mejor modelo según los `training_args`.

*   `trainer.evaluate()`:
    Después de que el entrenamiento ha finalizado, esta línea evalúa el modelo (normalmente el mejor modelo guardado si `load_best_model_at_end=True`) en el `eval_dataset` (conjunto de validación) y muestra un resumen de las métricas de rendimiento.

## **9. Prueba del modelo en nuevos textos**

Ahora podemos hacer predicciones en nuevos textos para evaluar el modelo.

In [41]:
# Evaluate the model on the test set and print the classification report

test_results = trainer.predict(test_data)
predictions = test_results.predictions.argmax(axis=-1)
labels = test_results.label_ids

print(predictions)
#print(labels)

print(classification_report(labels, predictions))

[0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0
 1 1 0 0 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 1
 1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0
 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 1 1 1
 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0
 0 1 0 1 1 1 1 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 1 0 0 0 0
 1 1 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1
 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 1 0
 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 0 0 1 0
 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 1 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0
 0 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 1
 0 0 0 0 1 0 1 1 0 1 0 0 

## **9. Prueba del modelo en nuevos textos**

En esta sección, se utiliza el modelo entrenado para hacer predicciones en el conjunto de prueba (`test_data`) y se evalúa su rendimiento final utilizando un informe de clasificación detallado.

*   `test_results = trainer.predict(test_data)`:
    *   El método `trainer.predict()` se utiliza para obtener las predicciones del modelo sobre el `test_data`.
    *   Este método devuelve un objeto que contiene las `predictions` (las salidas del modelo, a menudo logits) y las `label_ids` (las etiquetas reales del conjunto de prueba).

*   `predictions = test_results.predictions.argmax(axis=-1)`:
    *   Las `predictions` obtenidas de `trainer.predict()` son típicamente los *logits* (valores brutos antes de la función de activación final) para cada clase.
    *   `.argmax(axis=-1)` se aplica para convertir estos logits en las etiquetas de clase predichas, seleccionando el índice (la clase) con el valor más alto.

*   `labels = test_results.label_ids`:
    *   Esta línea simplemente asigna las etiquetas verdaderas del conjunto de prueba a la variable `labels` para facilitar su uso en la evaluación.

*   `print(predictions)`:
    *   Imprime el array de las etiquetas predichas por el modelo para el conjunto de prueba.

*   `print(classification_report(labels, predictions))`:
    *   Finalmente, se utiliza la función `classification_report` de `sklearn.metrics` para generar un informe de evaluación completo.
    *   Este informe incluye métricas clave como: `precision`, `recall`, `f1-score` y `support` (el número de instancias de cada clase) para cada clase, así como `accuracy` (precisión general), `macro avg` (promedio no ponderado de las métricas por clase) y `weighted avg` (promedio ponderado por el número de instancias por clase).
    *   Este informe es fundamental para entender el rendimiento del modelo en el conjunto de prueba, especialmente en problemas con desequilibrio de clases, donde la precisión simple puede ser engañosa.